In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.formula.api as smf

In [93]:
train_df = pd.read_csv("option_train.csv")
train_df.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420,0.341270,0.03013,Under
1,0.125000,427.015526,465,0.166667,0.03126,Over
2,20.691244,427.762336,415,0.265873,0.03116,Under
3,1.035002,451.711658,460,0.063492,0.02972,Over
4,39.553020,446.718974,410,0.166667,0.02962,Under


## Process Data

- split into train and validation sets
- normalize X-data (data did not appear gaussian)
- build standardized pandas DF for statsmodels ols regression summary

In [99]:
#create training and validation sets from df_train
train_nums = train_df.select_dtypes(["float64","int64"]) #not sure if allowed to use BS as a feature

y=train_nums["Value"].values
X=train_nums.drop("Value",axis=1).values

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.20,random_state = 1)


In [100]:
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_val_norm = mms.transform(X_val)

In [101]:
def normed_df(X,y):
    df1 = pd.DataFrame(X,columns = train_nums.columns[1:])
    df2 = pd.DataFrame(y,columns = ["Value"])
    df3 = pd.concat([df2,df1],axis=1)
    return df3

train_df_norm = normed_df(X_train_norm,y_train)
val_df_norm = normed_df(X_val_norm,y_val)

### Run regression models

Started with slr using K since it had the strongest linear relationship of any feature with value.

In [6]:
result1 = smf.ols('Value ~ K', data=train_df_norm).fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     4849.
Date:                Sat, 27 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:01:43   Log-Likelihood:                -4421.0
No. Observations:                1344   AIC:                             8846.
Df Residuals:                    1342   BIC:                             8856.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     48.2368      0.510     94.514      0.000      47.236      49.238
K            -65.6902      0.943    -69.635      0.000     -67.541     -63.840
==============================================================================
Omnibus:                       17.326   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.705
Skew:                           0.189   Prob(JB):                     0.000641
Kurtosis:                       2.655   Cond. No.                         6.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Full Model** <br>
Run full model using all features

In [7]:
feats = ' + '.join(train_df_norm.columns[1:])
result2 = smf.ols('Value ~' + feats,data=train_df_norm).fit()       
result2.summary()
           

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     3415.
Date:                Sat, 27 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:01:43   Log-Likelihood:                -3824.9
No. Observations:                1344   AIC:                             7660.
Df Residuals:                    1339   BIC:                             7686.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.0066      0.530     67.902      0.000      34.966      37.047
S             18.8558      0.529     35.641      0.000      17.818      19.894
K            -73.6662      0.637   -115.687      0.000     -74.915     -72.417
tau           12.4906      0.456     27.412      0.000      11.597      13.384
r              1.2907      0.545      2.368      0.018       0.221       2.360
==============================================================================
Omnibus:                      123.869   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              159.197
Skew:                           0.843   Prob(JB):                     2.70e-35
Kurtosis:                       3.005   Cond. No.                         9.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
result3 = smf.ols('Value ~ S + K + tau +r ',data=train_df_norm).fit()       
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     3415.
Date:                Sat, 27 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:16:02   Log-Likelihood:                -3824.9
No. Observations:                1344   AIC:                             7660.
Df Residuals:                    1339   BIC:                             7686.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.0066      0.530     67.902      0.000      34.966      37.047
S             18.8558      0.529     35.641      0.000      17.818      19.894
K            -73.6662      0.637   -115.687      0.000     -74.915     -72.417
tau           12.4906      0.456     27.412      0.000      11.597      13.384
r              1.2907      0.545      2.368      0.018       0.221       2.360
==============================================================================
Omnibus:                      123.869   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              159.197
Skew:                           0.843   Prob(JB):                     2.70e-35
Kurtosis:                       3.005   Cond. No.                         9.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### VIF (need to check with prof about this)

In [9]:
#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
[variance_inflation_factor(X_train_norm, j) for j in range(4)]

[4.3987363045647845, 7.203322349187848, 4.275181239998071, 2.182580432817126]

**Interaction Models** <br>

Currently the "best" model is using K, S and Ktau + Stau

In [10]:
result4 = smf.ols('Value ~ K + S + K*tau + S*tau ',data=train_df_norm).fit()       
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.918
Method:                 Least Squares   F-statistic:                     2991.
Date:                Sat, 27 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:01:43   Log-Likelihood:                -3768.8
No. Observations:                1344   AIC:                             7550.
Df Residuals:                    1338   BIC:                             7581.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     42.5320      0.874     48.664      0.000      40.817      44.247
K            -89.8860      1.587    -56.641      0.000     -92.999     -86.773
S             21.9344      0.999     21.955      0.000      19.974      23.894
tau            2.4650      1.487      1.658      0.098      -0.452       5.382
K:tau         27.7961      2.531     10.981      0.000      22.831      32.762
S:tau         -6.9999      1.799     -3.891      0.000     -10.530      -3.470
==============================================================================
Omnibus:                      139.009   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              182.425
Skew:                           0.893   Prob(JB):                     2.44e-40
Kurtosis:                       3.255   Cond. No.                         41.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
result5 = smf.ols('Value ~ K + S*tau + K*tau + K*r + K*S',data = train_df_norm).fit()
result5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     1942.
Date:                Sun, 28 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:49:02   Log-Likelihood:                -3743.8
No. Observations:                1344   AIC:                             7506.
Df Residuals:                    1335   BIC:                             7552.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.6150      1.344     26.493      0.000      32.978      38.252
K            -77.1232      2.550    -30.246      0.000     -82.125     -72.121
S             29.3575      1.656     17.727      0.000      26.109      32.606
tau            2.1444      1.464      1.465      0.143      -0.727       5.015
S:tau         -6.1354      1.780     -3.447      0.001      -9.627      -2.644
K:tau         27.4790      2.501     10.988      0.000      22.573      32.385
r              9.5418      1.378      6.926      0.000       6.839      12.244
K:r          -16.9988      2.685     -6.332      0.000     -22.265     -11.732
K:S          -13.8675      2.709     -5.120      0.000     -19.181      -8.554
==============================================================================
Omnibus:                      122.200   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.128
Skew:                           0.834   Prob(JB):                     1.25e-34
Kurtosis:                       3.085   Cond. No.                         61.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**A few models I tried**

| Formula     | Adj-R2      |    AIC    |
| ----------- | ----------- | --------- |
| Value ~ K | .783 | 8846 |
| Value ~ S + K + tau | .910 | 7663 |
| Value ~ S + K + tau + r | .910 | 7550 | 
| Value ~ K + S\*tau + K\*r + K\*S | 9.13  | 7620 |
| Value ~ K + S + K\*tau + S\*tau  | 9.18  | 7550 |
| Value ~ K +  K\*tau + K\*r + K\*S | .920 | 7516 |
| Value ~ K + S\*tau + K\*tau + K\*r + K\*S | .920 | 7506|


## Notes

1. K seems to be the best variable so far which makes since from scatter plots in EDA file.
2. Can look at transformation (x^2) for K to see if that improves things. 
3. S, K, tau provide the best adjusted R2
4. investigate interaction terms 

### Sklearn and Out-of-Sample R2

**tasks**
- build interaction dataFrame then run in sklearn (polynomialfearure)
- get out-of-sample R2 on validation set

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

lm = LinearRegression()

In [112]:
#generating interaction terms
#build interaction dataframe
def build_interaction(arr):
    poly = PolynomialFeatures(interaction_only=True,include_bias = False)
    X_int = poly.fit_transform(arr)
    interaction_headers = poly.get_feature_names(train_df_norm.columns[1:])
    X_int_df = pd.DataFrame(X_int,columns = interaction_headers) 
    return X_int_df

X_train_df_int = build_interaction(X_train_norm)
X_val_df_int = build_interaction(X_val_norm)

In [114]:
#build df
X_train_df_int.head()

,S,K,tau,r,S K,S tau,S r,K tau,K r,tau r
0,0.243162,0.80,0.632653,0.219409,0.194529,0.153837,0.053352,0.506122,0.175527,0.138810
1,0.606384,0.72,0.408163,0.481013,0.436597,0.247504,0.291679,0.293878,0.346329,0.196332
2,0.621094,0.56,0.877551,0.219409,0.347812,0.545041,0.136274,0.491429,0.122869,0.192543
3,0.120179,0.48,0.061224,0.738397,0.057686,0.007358,0.088740,0.029388,0.354430,0.045208
4,0.237539,0.64,0.887755,0.434599,0.152025,0.210876,0.103234,0.568163,0.278143,0.385818


In [119]:
#build function to calculate R2 values for multiple models (in and out R2)
def run_model(terms,train_df,y_train,val_df,y_val):
    X_train = train_df[terms].values
    X_val = val_df[terms].values

    lm.fit(X_train,y_train)
    in_sample_r2 = lm.score(X_train,y_train)
    out_of_sample_r2 = lm.score(X_val,y_val)
    return in_sample_r2, out_of_sample_r2

In [122]:
#Value ~ K +  K\*tau + K\*r + K\*S
in_r2, out_r2 = run_model(["K","K tau","K r","S K"],X_train_df_int,y_train,X_val_df_int,y_val)
print(f'In-sample-R2: {in_r2}\nOut-of-Sample-R2: {out_r2}')

In-sample-R2: 0.8995645266763367
Out-of-Sample-R2: 0.8871221104411545


In [124]:
#Value ~ K + S*tau + K*tau + K*r + K*S
in_r2, out_r2 = run_model(["K","S tau" , "K tau","K r","S K"],X_train_df_int,y_train,X_val_df_int,y_val)
print(f'In-sample-R2: {in_r2}\nOut-of-Sample-R2: {out_r2}')

In-sample-R2: 0.9010737442477936
Out-of-Sample-R2: 0.8932470412927223


## Outliers

**tasks**
- run again but drop three outliers
- train model on entire dataset before final predictions

## KNN Regression

**tasks**

- run interaction models from above using same data with KNN Regression model 
- see if more performant
- run with outliers then again without outliers